In [1]:
from database_utils import DatabaseConnector
from data_extraction import DataExtractor
from data_cleaning import DataCleaning
from dateutil.parser import parse
import pandas as pd
import numpy as np

In [2]:
#intializing engine
connector = DatabaseConnector()
engine1, engine2 = connector.init_db_engine()

In [3]:
#extracting the card details from the cloud
extractor = DataExtractor()
pdf_path = 'https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf'
extracted_card_df = extractor.retrieve_pdf_data(pdf_path)
extracted_card_df.info()


c:\Users\mohdi\Miniconda3\envs\mrdc\Lib\site-packages\tabula\io.py:1045: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15309 entries, 0 to 15308
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   card_number             15309 non-null  object
 1   expiry_date             15309 non-null  object
 2   card_provider           15309 non-null  object
 3   date_payment_confirmed  15309 non-null  object
dtypes: object(4)
memory usage: 478.5+ KB


In [ ]:
#15284

In [4]:
print(extracted_card_df.date_payment_confirmed.head())
print(f"Value count is: \n{extracted_card_df.date_payment_confirmed.value_counts()}")

0    2015-11-25
1    2001-06-18
2    2000-12-26
3    2011-02-12
4    1997-03-13
Name: date_payment_confirmed, dtype: object
Value count is: 
date_payment_confirmed
NULL          11
2009-12-20     8
1998-02-25     8
2017-03-31     7
2000-09-26     7
              ..
2016-09-18     1
2007-01-11     1
2021-07-09     1
2015-05-24     1
2009-02-04     1
Name: count, Length: 8270, dtype: int64


In [5]:

extracted_card_df.dropna(subset=['date_payment_confirmed'], inplace=True)
#This doesnt drop nulls because they are not recognised by pandas because they are string
print(extracted_card_df.date_payment_confirmed.iloc[377])



NULL


In [20]:
cols = ['NULL']
mask = extracted_card_df.date_payment_confirmed.isin(cols)
subset_df = extracted_card_df[mask]
extracted_card_df1 = subset_df.dropna( how = 'any', inplace= True)

C:\Users\mohdi\AppData\Local\Temp\ipykernel_9508\3112648347.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extracted_card_df1 = subset_df.dropna( how = 'any', inplace= True)


In [7]:
#we have to manually  handle these NULL values in each column
extracted_card_df.date_payment_confirmed = extracted_card_df.date_payment_confirmed.replace('NULL', pd.NaT)
extracted_card_df.expiry_date = extracted_card_df.expiry_date.replace('NULL', pd.NaT)
extracted_card_df.card_provider = extracted_card_df.card_provider.replace('NULL', np.NaN)
extracted_card_df.card_number = extracted_card_df.card_number.replace('NULL', np.NaN)
print(extracted_card_df.date_payment_confirmed.iloc[377])

NaT


In [8]:
extracted_card_df.date_payment_confirmed.iloc[6667]

'December 2000 01'

In [9]:
#from this it is clear the entire row is gibberish, hence we need to filter it out by a column which has less types and delete the entire row
extracted_card_df.head(828)

,card_number,expiry_date,card_provider,date_payment_confirmed
0,30060773296197,09/26,Diners Club / Carte Blanche,2015-11-25
1,349624180933183,10/23,American Express,2001-06-18
2,3529023891650490,06/23,JCB 16 digit,2000-12-26
3,213142929492281,09/27,JCB 15 digit,2011-02-12
4,502067329974,10/25,Maestro,1997-03-13
...,...,...,...,...
823,4918636593179,02/27,VISA 13 digit,2019-05-28
824,4787915516973,02/24,VISA 13 digit,2003-01-04
825,4004698644110600,06/27,VISA 16 digit,1993-10-11
826,180092318240446,03/23,JCB 15 digit,1994-05-11


In [10]:
extracted_card_df.card_provider.value_counts()

card_provider
VISA 16 digit                  2426
JCB 16 digit                   2403
VISA 13 digit                  1382
JCB 15 digit                   1363
VISA 19 digit                  1349
Diners Club / Carte Blanche    1312
American Express               1297
Maestro                        1281
Discover                       1260
Mastercard                     1211
OGJTXI6X1H                        1
BU9U947ZGV                        1
UA07L7EILH                        1
XGZBYBYGUW                        1
DLWF2HANZF                        1
1M38DYQTZV                        1
JRPRLPIBZ2                        1
DE488ORDXY                        1
5CJH7ABGDR                        1
JCQMU8FN85                        1
TS8A81WFXV                        1
WJVMUO4QX6                        1
NB71VBAHJE                        1
5MFWFBZRM9                        1
Name: count, dtype: int64

In [11]:
card_details_to_keep = ['VISA 16 digit','JCB 16 digit' ,'VISA 13 digit' 'JCB 15 digit ', 'VISA 19 digit', 'Diners Club / Carte Blanche', 'American Express', 'Maestro', 'Discover', 'Mastercard' ]
extracted_card_df.card_provider = extracted_card_df.card_provider[extracted_card_df.card_provider.isin(card_details_to_keep)]

In [12]:
extracted_card_df.card_provider.value_counts()

card_provider
VISA 16 digit                  2426
JCB 16 digit                   2403
VISA 19 digit                  1349
Diners Club / Carte Blanche    1312
American Express               1297
Maestro                        1281
Discover                       1260
Mastercard                     1211
Name: count, dtype: int64

In [14]:
print(extracted_card_df.card_provider.head(50))

0     Diners Club / Carte Blanche
1                American Express
2                    JCB 16 digit
3                             NaN
4                         Maestro
5                    JCB 16 digit
6                American Express
7                      Mastercard
8                        Discover
9                         Maestro
10                       Discover
11                       Discover
12                   JCB 16 digit
13               American Express
14                   JCB 16 digit
15                     Mastercard
16                  VISA 19 digit
17                       Discover
18                  VISA 19 digit
19                   JCB 16 digit
20                            NaN
21                   JCB 16 digit
22                  VISA 16 digit
23                  VISA 19 digit
24                       Discover
25               American Express
26                   JCB 16 digit
27                  VISA 16 digit
28                   JCB 16 digit
29    Diners C

In [28]:
extracted_card_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15309 entries, 0 to 15308
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   card_number             15298 non-null  object
 1   expiry_date             15298 non-null  object
 2   card_provider           12539 non-null  object
 3   date_payment_confirmed  15298 non-null  object
dtypes: object(4)
memory usage: 478.5+ KB
